# ml/evidently_drift_job.ipynb
Automated data & model drift report for Plutus models using Evidently.


In [ ]:
import os, json, pandas as pd
from datetime import datetime
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, RegressionPreset, ClassificationPreset

# CONFIG
MODEL_TYPE = os.getenv('MODEL_TYPE', 'classification')  # or 'regression'
REF_PATH = os.getenv('REF_PATH', 'data/reference.csv')
CUR_PATH = os.getenv('CUR_PATH', 'data/current.csv')
TARGET_COL = os.getenv('TARGET_COL', 'label')
PRED_COL = os.getenv('PRED_COL', 'prediction')
OUT_DIR = os.getenv('OUT_DIR', 'reports')
os.makedirs(OUT_DIR, exist_ok=True)

ref = pd.read_csv(REF_PATH)
cur = pd.read_csv(CUR_PATH)

if MODEL_TYPE == 'classification':
    metrics = [DataDriftPreset(), ClassificationPreset()]
else:
    metrics = [DataDriftPreset(), RegressionPreset()]

report = Report(metrics=metrics)
report.run(reference_data=ref, current_data=cur)

ts = datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')
html_path = os.path.join(OUT_DIR, f'drift_report_{ts}.html')
json_path = os.path.join(OUT_DIR, f'drift_report_{ts}.json')
report.save_html(html_path)
with open(json_path, 'w') as f:
    json.dump(report.as_dict(), f)
print('WROTE', html_path, json_path)
